In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
####################################################################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 
# 8/23/23 We want to add:
#     Take into account the RELATIVE STIM LOCATION within every barrel. Does velocity depend on distance from stimulating location?
#            New column: distance (um) from stim loc to relevant barrel pair boundary (1:2)
#            New column: distance (um) from stim loc to relevant barrel pair boundary (2:3)
#
#     METHOD
#         To calculate stim location to boundary, take these files produced by MovieMaker2.ipynb:
#             electrode.dat
#             corners_layer.dat
#             corners_barrel.dat
#         Calculate: Distance to lower barrel, distance to higher barrel,
#                    Distance to lower layer, distance to higher layer
#                       (if applicable for each)
####################################################################################

In [6]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
data_dir = 'C:/Users/jjudg/Downloads/'
cell_type_prefix = 'scnn1a'
csv_prefix = cell_type_prefix + '_all_slices-final.csv'

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Slice Edge,Medial Dir,Slice Location,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,...,abs(delta 1),abs(delta 2),Distance to Barrel 1,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2,Distance to Barrel A,Distance to Barrel B,Distance to Layer nA,Distance to Layer nB
0,2/8/2023,02_01_02,NaN,NaN,NaN,L4,47.7369,49.4509,49.8291,1.55191,...,NaN,NaN,NaN,NaN,NaN,NaN,215.228351,174.000000,180.093736,191.535134
1,2/10/2023,04_01_07,NaN,NaN,NaN,L4,52.7236,49.9505,51.1170,6.20749,...,2.7731,NaN,NaN,NaN,NaN,NaN,151.086327,213.337695,149.891961,202.354632
2,2/14/2023,02_01_01,NaN,NaN,NaN,L4,47.5447,49.7343,49.9818,2.20928,...,NaN,NaN,NaN,NaN,NaN,NaN,198.974687,232.730513,132.022414,229.523778
3,2/14/2023,03_01_03,NaN,NaN,NaN,L4,51.1559,50.3250,51.2512,4.77084,...,NaN,0.9262,NaN,NaN,NaN,NaN,228.000000,162.000000,159.011987,199.155462
4,2/28/2023,02_01_01,NaN,NaN,NaN,L5,50.3009,50.8406,49.3457,2.94039,...,NaN,1.4949,NaN,NaN,NaN,NaN,44.346286,328.107622,171.794234,154.795349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,10/19/2023,05_06_02,down/right,down/left,Anterior,L2/3,52.5402,51.2761,54.6519,5.21410,...,1.2641,NaN,NaN,NaN,NaN,NaN,24.997803,181.431965,314.744870,103.797327
170,10/19/2023,06_01_01,down,right,Posterior,L4,50.6833,50.3330,49.7638,10.10250,...,0.3503,NaN,NaN,NaN,NaN,NaN,118.898651,202.376373,288.667605,69.296479
171,10/19/2023,06_02_02,down,right,Posterior,L4,50.3022,51.0678,52.1046,27.20220,...,0.7656,1.0368,NaN,NaN,NaN,NaN,151.005549,215.305522,75.027164,306.897033
172,10/19/2023,06_03_01,down,right,Posterior,L2/3,52.3050,51.7377,50.8937,8.65164,...,0.5673,NaN,NaN,NaN,NaN,NaN,322.923386,22.555042,70.935246,350.093066


In [13]:
# filter out rows marked Exclude 
df2 = df[df["Exclude Reason"].str.len() < 1]
print(df2)

for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    print("Barrel pairs in", stim_loc)
    # count number of barrel pairs with no signal crossing
    n_barrel_no_cross = len(df2[df2['# no crossings 2/3'] > 0])
    n_barrel_no_cross += len(df2[df2['# no crossings 1/2'] > 0])
    print("\t", n_barrel_no_cross, "no crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with signal crossing
    n_barrel_cross = len(df2[df2['# crossings 2/3'] > 0])
    n_barrel_cross += len(df2[df2['# crossings 1/2'] > 0])
    print("\t", n_barrel_cross, "crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with no signal
    n_barrel_none = len(df2) * 2 - n_barrel_cross - n_barrel_no_cross
    print("\t", n_barrel_none, "no signal out of", len(df2) * 2, "in layer", stim_loc)

Empty DataFrame
Columns: [Date, Slice/Loc/Rec, Slice Edge, Medial Dir, Slice Location, Stim Location, L4 barrel 1 Latency, L4 barrel 2 Latency, L4 barrel 3 Latency, L4 barrel 1 SNR, L4 barrel 2 SNR, L4 barrel 3 SNR, Exclude Reason, # no crossings 1/2, # crossings 1/2, # no crossings 2/3, # crossings 2/3, ROI jpg, Age, Infusion, Sex, Stim (uA), Delta latency 1 (barrel 1 - barrel 2), Delta latency 2 (barrel 3 - barrel 2), abs(delta 1), abs(delta 2), Distance to Barrel 1, Distance to Barrel 2, Distance to Layer n1, Distance to Layer n2, Distance to Barrel A, Distance to Barrel B, Distance to Layer nA, Distance to Layer nB]
Index: []

[0 rows x 34 columns]
Barrel pairs in L2/3
	 19 no crossing out of 122 in layer L2/3
	 46 crossing out of 122 in layer L2/3
	 57 no signal out of 122 in layer L2/3
Barrel pairs in L4
	 68 no crossing out of 194 in layer L4
	 35 crossing out of 194 in layer L4
	 91 no signal out of 194 in layer L4
Barrel pairs in L5
	 10 no crossing out of 28 in layer L5
	 7 c

In [8]:
df['abs(delta 1)'] = np.abs(df['Delta latency 1 (barrel 1 - barrel 2)'])
df['abs(delta 2)'] = np.abs(df['Delta latency 2 (barrel 3 - barrel 2)'])



for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    dfd1 = df2['abs(delta 1)'].dropna()
    dfd2 = df2['abs(delta 2)'].dropna()

    print("Latency between barrel pairs in", stim_loc)
    dfds = np.concatenate([dfd1, dfd2])
    print("\t", 
          str(np.mean(dfds))[:5], 
          "+/-", 
          str(np.std(dfds))[:5], 
          "(n =", 
          str(len(dfds)) + ")")
    print("\t", dfds)

Latency between barrel pairs in L2/3
	 0.545 +/- 0.417 (n = 5)
	 [0.2138 0.2299 0.7214 0.2692 1.2911]
Latency between barrel pairs in L4
	 1.188 +/- 0.874 (n = 8)
	 [2.7731 0.0717 1.222  0.9262 0.957  1.3192 0.0676 2.1705]
Latency between barrel pairs in L5
	 1.043 +/- 0.473 (n = 2)
	 [0.5704 1.5165]


In [11]:
# write the pandas dataframe back to csv
csv_filename = cell_type_prefix + '_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [12]:
df

,Date,Slice/Loc/Rec,Slice Location,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,...,abs(delta 1),abs(delta 2),Distance to Barrel 1,Distance to Barrel 2,Distance to Layer n1,Distance to Layer n2,Distance to Barrel A,Distance to Barrel B,Distance to Layer nA,Distance to Layer nB
0,2/8/2023,02_01_02,NaN,L4,47.7369,49.4509,49.8291,1.551910,4.440250,3.03586,...,NaN,NaN,NaN,NaN,NaN,NaN,215.228351,174.000000,180.093736,191.535134
1,2/10/2023,04_01_07,NaN,L4,52.7236,49.9505,51.1170,6.207490,13.131100,2.27455,...,2.7731,NaN,NaN,NaN,NaN,NaN,151.086327,213.337695,149.891961,202.354632
2,2/14/2023,02_01_01,NaN,L4,47.5447,49.7343,49.9818,2.209280,2.558190,5.16311,...,NaN,NaN,NaN,NaN,NaN,NaN,198.974687,232.730513,132.022414,229.523778
3,2/14/2023,03_01_03,NaN,L4,51.1559,50.3250,51.2512,4.770840,12.652100,6.62977,...,NaN,0.9262,NaN,NaN,NaN,NaN,228.000000,162.000000,159.011987,199.155462
4,2/28/2023,04_01_06,NaN,L2/3,55.0473,52.3564,51.1973,1.274260,3.233650,4.78556,...,NaN,NaN,NaN,NaN,NaN,NaN,189.026639,198.000000,206.933090,142.367160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,9/7/2023,02_04_03,Posterior,L2/3,47.6409,48.9078,49.2190,2.692310,2.403120,1.92779,...,NaN,NaN,NaN,NaN,NaN,NaN,64.400289,94.218365,186.287123,34.551724
84,9/7/2023,03_01_06,Anterior,L4,47.4999,48.3735,47.3385,2.259410,3.250860,2.56239,...,NaN,NaN,NaN,NaN,NaN,NaN,132.012904,31.258292,299.960273,14.797950
85,9/7/2023,03_02_03,Anterior,L4,49.3103,47.0878,41.8927,1.618110,1.184490,1.03614,...,NaN,NaN,NaN,NaN,NaN,NaN,128.598914,28.784061,160.091531,33.369509
86,9/7/2023,03_03_05,Anterior,L4,48.4769,50.3454,50.5443,0.762933,0.878104,1.19032,...,NaN,NaN,NaN,NaN,NaN,NaN,53.210871,69.000183,162.000000,32.386070
